In [5]:
import pandas as pd
import os
import glob
import csv

# Папка с файлами
current_folder = os.getcwd()
print(f"Текущая папка: {current_folder}")

# Находим все CSV файлы
csv_files = glob.glob(os.path.join(current_folder, "*.csv"))

if not csv_files:
    print("Не найдено CSV файлов в текущей папке")
    exit()

print(f"Найдено файлов: {len(csv_files)}")

# Создаем папку для исправленных файлов
fixed_folder = os.path.join(current_folder, "fixed")
if not os.path.exists(fixed_folder):
    os.makedirs(fixed_folder)

# Кодировки для перебора
encodings_to_try = [
    'utf-8-sig',     # UTF-8 с BOM
    'windows-1251',  # Windows кириллица
    'cp1251',        # альтернатива windows-1251
    'cp866',         # DOS кириллица
    'koi8-r',        # Unix кириллица
    'iso-8859-5',    # Latin/Cyrillic
    'utf-8',         # UTF-8 без BOM
]

# Разделители для перебора
delimiters_to_try = [',', ';', '\t', '|', ' ']

success_count = 0

for csv_file in csv_files:
    filename = os.path.basename(csv_file)
    print(f"\n{'='*60}")
    print(f"Обработка файла: {filename}")
    
    df = None
    used_encoding = None
    used_delimiter = None
    
    # Пробуем разные комбинации кодировок и разделителей
    for encoding in encodings_to_try:
        for delimiter in delimiters_to_try:
            try:
                # Пробуем прочитать с текущей комбинацией
                df = pd.read_csv(
                    csv_file, 
                    encoding=encoding, 
                    delimiter=delimiter,
                    on_bad_lines='skip',  # Пропускаем плохие строки
                    engine='python',       # Более гибкий парсер
                    quoting=csv.QUOTE_MINIMAL
                )
                
                # Проверяем, что получили разумное количество столбцов
                if len(df.columns) > 1:  # Если больше 1 столбца - вероятно правильный разделитель
                    used_encoding = encoding
                    used_delimiter = delimiter
                    print(f"  ✓ Удалось прочитать:")
                    print(f"     Кодировка: {encoding}")
                    print(f"     Разделитель: {repr(delimiter)}")
                    print(f"     Столбцов: {len(df.columns)}, Строк: {len(df)}")
                    break
            except Exception as e:
                continue
        
        if df is not None:
            break
    
    # Если не получилось с pandas, пробуем более простой способ
    if df is None:
        print(f"  ✗ Pandas не смог прочитать файл")
        print(f"  Пробую альтернативный способ чтения...")
        
        # Пробуем определить кодировку и разделитель вручную
        for encoding in encodings_to_try:
            try:
                # Читаем файл как текст
                with open(csv_file, 'r', encoding=encoding) as f:
                    lines = f.readlines()
                
                if len(lines) > 0:
                    # Анализируем первую строку для определения разделителя
                    first_line = lines[0]
                    
                    # Считаем частоту разных символов-разделителей
                    delimiter_counts = {delim: first_line.count(delim) for delim in delimiters_to_try}
                    
                    # Выбираем самый частый разделитель
                    best_delimiter = max(delimiter_counts.items(), key=lambda x: x[1])
                    
                    if best_delimiter[1] > 0:  # Если нашли разделитель
                        print(f"  ✓ Определено вручную:")
                        print(f"     Кодировка: {encoding}")
                        print(f"     Разделитель: {repr(best_delimiter[0])} (встречается {best_delimiter[1]} раз)")
                        
                        # Пробуем прочитать с этими параметрами
                        try:
                            df = pd.read_csv(
                                csv_file, 
                                encoding=encoding, 
                                delimiter=best_delimiter[0],
                                on_bad_lines='skip',
                                engine='python'
                            )
                            used_encoding = encoding
                            used_delimiter = best_delimiter[0]
                        except Exception as e:
                            continue
            except Exception as e:
                continue
    
    # Если все еще не получилось, пробуем прочитать как фиксированную ширину
    if df is None:
        print(f"  ✗ Не удалось определить структуру файла")
        print(f"  Пробую чтение как фиксированной ширины...")
        
        try:
            # Читаем весь файл как текст
            for encoding in encodings_to_try:
                try:
                    with open(csv_file, 'r', encoding=encoding) as f:
                        content = f.read()
                    
                    if len(content) > 0:
                        # Разбиваем по строкам
                        lines = content.strip().split('\n')
                        
                        # Создаем DataFrame из строк
                        data = []
                        for line in lines:
                            # Разбиваем по пробелам или другим возможным разделителям
                            parts = line.split()
                            if len(parts) > 0:
                                data.append(parts)
                        
                        if data:
                            # Создаем DataFrame с максимальным количеством столбцов
                            max_cols = max(len(row) for row in data)
                            columns = [f'col_{i}' for i in range(max_cols)]
                            
                            # Выравниваем все строки по максимальной длине
                            aligned_data = []
                            for row in data:
                                if len(row) < max_cols:
                                    row = row + [''] * (max_cols - len(row))
                                aligned_data.append(row)
                            
                            df = pd.DataFrame(aligned_data, columns=columns)
                            used_encoding = encoding
                            used_delimiter = 'whitespace'
                            print(f"  ✓ Прочитано как текст:")
                            print(f"     Кодировка: {encoding}")
                            print(f"     Разделитель: пробелы")
                            print(f"     Столбцов: {len(df.columns)}, Строк: {len(df)}")
                            break
                except Exception as e:
                    continue
        except Exception as e:
            print(f"  ✗ Ошибка при чтении как текст: {e}")
    
    if df is None:
        print(f"  ✗ Не удалось прочитать файл никаким способом")
        print(f"  Пропускаю файл: {filename}")
        continue
    
    # Сохраняем исправленный файл
    output_file = os.path.join(fixed_folder, filename)
    
    try:
        # Сохраняем в UTF-8 с BOM с разделителем-запятой
        df.to_csv(output_file, index=False, encoding='utf-8-sig')
        
        # Читаем обратно для проверки
        try:
            test_df = pd.read_csv(output_file, encoding='utf-8-sig', nrows=3)
            print(f"  ✓ Сохранен исправленный файл: {output_file}")
            print(f"     Проверка: {len(test_df.columns)} столбцов, {len(test_df)} строк")
            
            # Показываем первые несколько символов для проверки кириллицы
            if len(test_df) > 0:
                sample_text = ""
                for col in test_df.columns[:3]:  # Первые 3 столбца
                    if len(test_df[col]) > 0:
                        val = str(test_df[col].iloc[0])
                        sample_text += val[:30] + "... "
                
                # Проверяем на кириллицу
                cyrillic_chars = sum(1 for char in sample_text if 'а' <= char.lower() <= 'я')
                if cyrillic_chars > 0:
                    print(f"     Кириллица: ✓ обнаружена ({cyrillic_chars} символов)")
                    print(f"     Пример: {sample_text[:50]}...")
                else:
                    print(f"     Кириллица: ⚠ не обнаружена")
                    print(f"     Пример: {sample_text[:50]}...")
            
            success_count += 1
            
        except Exception as e:
            print(f"  ⚠ Сохранено, но ошибка проверки: {e}")
            
    except Exception as e:
        print(f"  ✗ Ошибка при сохранении: {e}")

print(f"\n{'='*60}")
print(f"ОБРАБОТКА ЗАВЕРШЕНА!")
print(f"Успешно обработано: {success_count} из {len(csv_files)} файлов")
print(f"Исправленные файлы сохранены в: {fixed_folder}")

if success_count > 0:
    print(f"\nПросмотр содержимого исправленных файлов:")
    fixed_files = glob.glob(os.path.join(fixed_folder, "*.csv"))
    for fixed_file in fixed_files:
        filename = os.path.basename(fixed_file)
        try:
            df = pd.read_csv(fixed_file, encoding='utf-8-sig', nrows=2)
            print(f"\n{filename}:")
            print(f"  Столбцы: {list(df.columns)}")
            print(f"  Строк: {len(df)}")
        except Exception as e:
            print(f"\n{filename}: ошибка чтения - {e}")

Текущая папка: d:\Projects\thesis\data\channels\archive
Найдено файлов: 43

Обработка файла: apps_channels.csv
  ✓ Удалось прочитать:
     Кодировка: utf-8-sig
     Разделитель: '\t'
     Столбцов: 9, Строк: 3806
  ✓ Сохранен исправленный файл: d:\Projects\thesis\data\channels\archive\fixed\apps_channels.csv
     Проверка: 9 столбцов, 3 строк
     Кириллица: ⚠ не обнаружена
     Пример: 1... apps... Global... ...

Обработка файла: art_channels.csv
  ✓ Удалось прочитать:
     Кодировка: utf-8-sig
     Разделитель: '\t'
     Столбцов: 9, Строк: 11253
  ✓ Сохранен исправленный файл: d:\Projects\thesis\data\channels\archive\fixed\art_channels.csv
     Проверка: 9 столбцов, 3 строк
     Кириллица: ⚠ не обнаружена
     Пример: 1... art... RU... ...

Обработка файла: babies_channels.csv
  ✓ Удалось прочитать:
     Кодировка: utf-8-sig
     Разделитель: '\t'
     Столбцов: 9, Строк: 7316
  ✓ Сохранен исправленный файл: d:\Projects\thesis\data\channels\archive\fixed\babies_channels.csv
     Про